In [1]:
import numpy as np
import torch
import pickle
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D, Dataset1D

In [2]:
dataset_name = '2_29_sine_dataset'
# training_seq = torch.rand(64, 32, 128) # features are normalized from 0 to 1
with open(f'./dataset/{dataset_name}/data.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [3]:
from torch.utils.data import Dataset, DataLoader
from multiprocessing import cpu_count

dl = DataLoader(dataset, batch_size = 32, shuffle = True, pin_memory = True, num_workers = cpu_count())

In [7]:
data =[x[0] for x in dl]
labels = [x[1] for x in dl]

In [7]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

In [8]:
dl = cycle(dl)

In [9]:
data = next(dl)

In [11]:
img, labels = data

In [13]:
labels

[tensor([0.4400, 0.6800, 0.2600, 0.5600, 0.2400, 0.9600, 0.8200, 0.5600, 0.4200,
         0.2600, 0.8000, 0.5800, 0.8800, 0.7800, 0.6000, 0.6800, 0.6000, 0.9600,
         0.5800, 0.5600, 0.2600, 0.5800, 0.3600, 0.3800, 0.8200, 0.5600, 0.3600,
         0.3200, 0.9800, 0.2000, 0.4800, 0.3600], dtype=torch.float64),
 tensor([1.1000, 3.9000, 2.1000, 1.8000, 1.9000, 2.1000, 2.1000, 2.1000, 4.5000,
         4.4000, 2.1000, 2.1000, 3.8000, 4.8000, 1.5000, 4.6000, 4.0000, 2.6000,
         2.7000, 4.8000, 2.4000, 3.4000, 4.2000, 3.0000, 4.8000, 2.4000, 2.0000,
         2.0000, 2.0000, 1.9000, 1.3000, 3.6000], dtype=torch.float64)]

In [10]:
dl.item(0)

AttributeError: 'generator' object has no attribute 'item'

In [32]:
for i, (data, labels) in enumerate(dl):
    print(np.array(data).shape, np.array(labels[0]).shape, np.array(labels[1]).shape)
    if i > 10:
        break

(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)
(32, 128) (32,) (32,)


In [28]:
np.array(data[1][1]).shape

(32,)

In [14]:
model = Unet1D(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 32
)

diffusion = GaussianDiffusion1D(
    model,
    seq_length = 128,
    timesteps = 1000,
    objective = 'pred_noise'
)
dataset_name = '2_29_sine_dataset'
# training_seq = torch.rand(64, 32, 128) # features are normalized from 0 to 1
with open(f'./dataset/{dataset_name}/data.pkl', 'rb') as f:
    dataset = pickle.load(f)
# dataset = Dataset1D(training_seq)  # this is just an example, but you can formulate your own Dataset and pass it into the `Trainer1D` below

# loss = diffusion(dataset)
# loss.backward()

# Or using trainer

trainer = Trainer1D(
    diffusion,
    dataset = dataset,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)
trainer.train()

# after a lot of training

# sampled_seq = diffusion.sample(batch_size = 4)
# sampled_seq.shape # (4, 32, 128)

: 